In [0]:
# ============================================================================
# Camada Gold - Análise Mercado Cartas Executivo - Magic: The Gathering
# Pipeline 100% PySpark DataFrame API
# Seguindo padrão Silver com Unity Catalog e merge incremental
# ============================================================================

# ============================================================================
# BIBLIOTECAS UTILIZADAS
# ============================================================================
import logging
from datetime import datetime
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import DeltaTable
from pyspark.sql.utils import AnalysisException

# ============================================================================
# FUNÇÕES UTILITÁRIAS
# ============================================================================
def get_secret(secret_name, default_value=None):
    """Obtém segredos do Databricks Secret Scope"""
    try:
        return dbutils.secrets.get(scope="mtg-pipeline", key=secret_name)
    except:
        if default_value is not None:
            print(f"Secret '{secret_name}' não encontrado, usando valor padrão")
            return default_value
        else:
            print(f"Secret obrigatório '{secret_name}' não encontrado")
            raise Exception(f"Required secret '{secret_name}' not configured")

def setup_unity_catalog(catalog, schema):
    """Configura Unity Catalog"""
    try:
        spark.sql(f"CREATE CATALOG IF NOT EXISTS {catalog}")
        spark.sql(f"USE CATALOG {catalog}")
        spark.sql(f"CREATE SCHEMA IF NOT EXISTS {schema}")
        spark.sql(f"USE SCHEMA {schema}")
        print(f"✅ Schema {catalog}.{schema} configurado com sucesso")
        return True
    except Exception as e:
        print(f"❌ Erro ao configurar Unity Catalog: {e}")
        return False

def load_to_gold_unity_incremental(df_final, catalog, schema, table_name, s3_gold_path):
    """Carrega dados na camada Gold com suporte a Unity Catalog"""
    delta_path = f"s3://{s3_gold_path}/{table_name}"
    full_table_name = f"{catalog}.{schema}.{table_name}"
    
    print(f"Salvando dados em: {delta_path}")
    print(f"Qtd linhas df_final: {df_final.count()}")
    
    try:
        # Sobrescrever para análises (dados agregados)
        df_final.write.format("delta") \
            .mode("overwrite") \
            .option("overwriteSchema", "true") \
            .partitionBy("DATA_REF") \
            .save(delta_path)
        
        # Criar/atualizar tabela Unity Catalog
        try:
            spark.sql(f"SELECT 1 FROM {full_table_name} LIMIT 1")
            print(f"ℹ️ Tabela Unity Catalog '{full_table_name}' já existe")
        except:
            spark.sql(f"""
                CREATE TABLE {full_table_name}
                USING DELTA
                LOCATION '{delta_path}'
            """)
            print(f"✅ Tabela Unity Catalog criada: {full_table_name}")
        
        print("✅ Dados salvos com sucesso!")
        
    except Exception as e:
        print(f"❌ Erro ao salvar dados: {e}")
        raise

# ============================================================================
# CONFIGURAÇÃO E VARIÁVEIS
# ============================================================================
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Usar secrets para configuração
CATALOG_NAME = get_secret("catalog_name")
SCHEMA_SILVER = "silver"
SCHEMA_GOLD = "gold"
TABLE_NAME = "TB_ANALISE_MERCADO_CARTAS_EXECUTIVO"
S3_BUCKET = get_secret("s3_bucket")
S3_GOLD_PREFIX = get_secret("s3_gold_prefix", "magic_the_gathering/gold")
S3_GOLD_PATH = f"{S3_BUCKET}/{S3_GOLD_PREFIX}"

# Configurar Unity Catalog
setup_unity_catalog(CATALOG_NAME, SCHEMA_GOLD)

# Inicialização Spark
spark = SparkSession.builder.getOrCreate()

# ============================================================================
# Leitura das Tabelas Silver
# ============================================================================
df_cards = spark.table(f"{CATALOG_NAME}.{SCHEMA_SILVER}.TB_FATO_SILVER_CARDS").alias("cards")
df_prices = spark.table(f"{CATALOG_NAME}.{SCHEMA_SILVER}.TB_FATO_SILVER_CARDPRICES").alias("prices")

# ============================================================================
# Join Cards + Prices pela chave comum
# ============================================================================
# Cards tem: NME_CARD, NME_SET, COD_SET, NME_CARD_TYPE, NME_RARITY, NME_ARTIST, DT_INGESTION
# Prices tem: NME_CARD, COD_SET, VLR_USD, VLR_EUR, VLR_TIX, NME_RARITY, DT_INGESTION
df = df_cards.join(df_prices, ["NME_CARD", "COD_SET"], "inner")

# ============================================================================
# Inclusão de colunas de interesse
# ============================================================================
# Usar DT_INGESTION da tabela cards (mais confiável para datas das cartas)
df = df.withColumn("DATA_REF", trunc(df_cards["DT_INGESTION"], "month"))

# ============================================================================
# KPIs por raridade, artista, coleção e tipo de carta
# ============================================================================
# JOIN resolve conflito de NME_RARITY: usar a coluna do cards (mais confiável)
df_gold = (
    df.groupBy("DATA_REF", "NME_SET", "NME_CARD_TYPE", "cards.NME_RARITY", "NME_ARTIST")
      .agg(
          sum("VLR_USD").alias("VALOR_TOTAL_MERCADO"),
          countDistinct("NME_CARD").alias("QTD_CARTAS_ATIVAS"),
          avg("VLR_USD").alias("VALOR_MEDIO_CARTA"),
          expr("percentile_approx(VLR_USD, 0.5)").alias("TICKET_MEDIANA")
      )
      .withColumnRenamed("cards.NME_RARITY", "NME_RARITY")  # Rename para remover prefixo
)

# ============================================================================
# Market share por set
# ============================================================================
from pyspark.sql.window import Window
window_set = Window.partitionBy("DATA_REF")
df_gold = df_gold.withColumn(
    "MARKET_SHARE_SET",
    col("VALOR_TOTAL_MERCADO") / sum("VALOR_TOTAL_MERCADO").over(window_set)
)

# ============================================================================
# Top N cartas por valorização/desvalorização (exemplo para N=3)
# ============================================================================
window_card = Window.partitionBy("NME_CARD").orderBy("DATA_REF")
df_val = (
    df.groupBy("DATA_REF", "NME_CARD")
      .agg(sum("VLR_USD").alias("VALOR_ATUAL"))
      .withColumn("VALOR_ANTERIOR", lag("VALOR_ATUAL").over(window_card))
      .withColumn("VARIACAO_PERC_VALOR", 
                  when(col("VALOR_ANTERIOR").isNotNull() & (col("VALOR_ANTERIOR") != 0),
                       (col("VALOR_ATUAL") - col("VALOR_ANTERIOR")) / col("VALOR_ANTERIOR"))
                  .otherwise(lit(None)))
)
window_risco = Window.partitionBy("DATA_REF").orderBy(col("VARIACAO_PERC_VALOR").desc())
df_val = df_val.withColumn("RANK_VALORIZACAO", row_number().over(window_risco))
df_val = df_val.withColumn(
    "TIPO_VARIACAO",
    when(col("VARIACAO_PERC_VALOR") > 0, "Valorização")
    .when(col("VARIACAO_PERC_VALOR") < 0, "Desvalorização")
    .otherwise("Estável")
)

# ============================================================================
# Escrita da Tabela de Análise Executiva (única saída deste script)
# ============================================================================
load_to_gold_unity_incremental(
    df_final=df_gold,
    catalog=CATALOG_NAME,
    schema=SCHEMA_GOLD,
    table_name=TABLE_NAME,
    s3_gold_path=S3_GOLD_PATH
)

# ============================================================================
# Fim do Script
# ============================================================================
print("TB_ANALISE_MERCADO_CARTAS_EXECUTIVO criado com sucesso!")
print("KPIs executivos consolidados!")
print("Padrão Silver aplicado: Unity Catalog + Secrets + Delta!")
print(f"Tabela criada: {CATALOG_NAME}.{SCHEMA_GOLD}.{TABLE_NAME}") 